# Pipeline processing using rsexecute workflows.

This notebook demonstrates the continuum imaging pipeline based on RASCIL functions wrapped
     up as SDP workflows using the rsexecute class.

In [ ]:
%matplotlib inline

import sys

results_dir = "/tmp/"

from matplotlib import pylab

pylab.rcParams["figure.figsize"] = (12.0, 12.0)
pylab.rcParams["image.cmap"] = "rainbow"

import numpy

from astropy.coordinates import SkyCoord
from astropy import units as u

from matplotlib import pyplot as plt

from rascil.data_models.polarisation import PolarisationFrame

from rascil.processing_components import (
    show_image,
    export_image_to_fits,
    qa_image,
    create_low_test_image_from_gleam,
    create_image_from_visibility,
    advise_wide_field,
)

from rascil.workflows import (
    invert_list_rsexecute_workflow,
    predict_list_rsexecute_workflow,
    simulate_list_rsexecute_workflow,
    continuum_imaging_skymodel_list_rsexecute_workflow,
)

from rascil.workflows.rsexecute.execution_support.rsexecute import rsexecute

import logging

log = logging.getLogger()
logging.info("Starting imaging-pipeline")
log.setLevel(logging.INFO)
log.addHandler(logging.StreamHandler(sys.stdout))

mpl_logger = logging.getLogger("matplotlib")
mpl_logger.setLevel(logging.WARNING)

Set up rsexecute to use dask.

In [ ]:
rsexecute.set_client(use_dask=True)

We create a graph to make the visibility. The parameter rmax determines the distance of the furthest antenna/stations 
used. 

Finally we compute the value of the graph, a list of visibility objects. These can then be scattered to the
cluster of workers.

In [ ]:
nfreqwin = 7
ntimes = 5
rmax = 750.0
frequency = numpy.linspace(1e8, 1.2e8, nfreqwin)
channel_bandwidth = numpy.array(nfreqwin * [frequency[1] - frequency[0]])
times = numpy.linspace(-numpy.pi / 3.0, numpy.pi / 3.0, ntimes)
phasecentre = SkyCoord(
    ra=+30.0 * u.deg, dec=-60.0 * u.deg, frame="icrs", equinox="J2000"
)

bvis_list = simulate_list_rsexecute_workflow(
    "LOWBD2",
    frequency=frequency,
    channel_bandwidth=channel_bandwidth,
    times=times,
    phasecentre=phasecentre,
    order="frequency",
    rmax=rmax,
)

print("%d elements in vis_list" % len(bvis_list))
log.info("About to make visibility")
bvis_list = rsexecute.compute(bvis_list, sync=True)

Find the recommended values for the wide field imaging.

In [ ]:
advice_low = advise_wide_field(bvis_list[0], guard_band_image=8.0, delA=0.02)
advice_high = advise_wide_field(bvis_list[-1], guard_band_image=8.0, delA=0.02)

npixel = advice_high["npixels2"]
cellsize = min(advice_low["cellsize"], advice_high["cellsize"])

Now make a graph to fill with a model drawn from GLEAM. There will be one model image
per frequency.

In [ ]:
gleam_model = [
    rsexecute.execute(create_low_test_image_from_gleam)(
        npixel=npixel,
        frequency=[frequency[f]],
        channel_bandwidth=[channel_bandwidth[f]],
        cellsize=cellsize,
        phasecentre=phasecentre,
        polarisation_frame=PolarisationFrame("stokesI"),
        flux_limit=1.0,
        applybeam=True,
    )
    for f, freq in enumerate(frequency)
]
log.info("About to make GLEAM model")
future_gleam_model = rsexecute.persist(gleam_model)

Now we can fill in the visibility values from the models, using a workflow.

In [ ]:
log.info("About to run predict to get predicted visibility")
future_vis_graph = rsexecute.scatter(bvis_list)
predicted_vislist = predict_list_rsexecute_workflow(
    future_vis_graph, gleam_model, context="ng"
)
predicted_vislist = rsexecute.compute(predicted_vislist, sync=True)
future_predicted_vislist = rsexecute.scatter(predicted_vislist)

Get the LSM. This is blank.

In [ ]:
model_list = [
    rsexecute.execute(create_image_from_visibility)(
        bvis_list[f],
        npixel=npixel,
        frequency=[frequency[f]],
        channel_bandwidth=[channel_bandwidth[f]],
        cellsize=cellsize,
        phasecentre=phasecentre,
        polarisation_frame=PolarisationFrame("stokesI"),
    )
    for f, freq in enumerate(frequency)
]

Make the graphs to calculate the dirty image and psf, but don't compute the values yet.

In [ ]:
dirty_list = invert_list_rsexecute_workflow(
    future_predicted_vislist, model_list, context="ng", dopsf=False
)
psf_list = invert_list_rsexecute_workflow(
    future_predicted_vislist, model_list, context="ng", dopsf=True
)

Make and compute the graph to make the dirty image

In [ ]:
log.info("About to run invert to get dirty image")
dirty_list = rsexecute.compute(dirty_list, sync=True)
dirty = dirty_list[0][0]
show_image(dirty, cm="Greys")
plt.show()

Finally we can try the continuum imaging pipeline. First construct the graph.

In [ ]:
centre = nfreqwin // 2

continuum_imaging_list = continuum_imaging_skymodel_list_rsexecute_workflow(
    future_predicted_vislist,
    model_imagelist=model_list,
    context="ng",
    algorithm="mmclean",
    scales=[0],
    niter=1000,
    fractional_threshold=0.1,
    threshold=0.003,
    nmoment=2,
    nmajor=5,
    gain=0.1,
    deconvolve_facets=4,
    deconvolve_overlap=32,
    deconvolve_taper="tukey",
    psf_support=64,
    window_shape="no_edge",
    do_wstacking=True,
)

Now execute the graph

In [ ]:
log.info("About to run continuum imaging")

centre = nfreqwin // 2
continuum_imaging_list = rsexecute.compute(continuum_imaging_list, sync=True)
residual = continuum_imaging_list[0][centre]
restored = continuum_imaging_list[1][centre]
deconvolved = continuum_imaging_list[2][centre].image


f = show_image(
    deconvolved, title="Clean image - no selfcal", cm="Greys", vmax=0.1, vmin=-0.01
)
print(qa_image(deconvolved, context="Clean image - no selfcal"))

plt.show()

f = show_image(
    restored, title="Restored clean image - no selfcal", cm="Greys", vmax=1.0, vmin=-0.1
)
print(qa_image(restored, context="Restored clean image - no selfcal"))
plt.show()
export_image_to_fits(
    restored, "%s/imaging-dask_continuum_imaging_restored.fits" % (results_dir)
)

f = show_image(residual[0], title="Residual clean image - no selfcal", cm="Greys")
print(qa_image(residual[0], context="Residual clean image - no selfcal"))
plt.show()
export_image_to_fits(
    residual[0], "%s/imaging-dask_continuum_imaging_residual.fits" % (results_dir)
)